In [1]:
%matplotlib widget

import tensorflow as tf
import os
import sys
import cv2
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import random

from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.layers import GlobalMaxPooling2D
from numpy.linalg import norm
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.svm import LinearSVC

In [2]:
DATASET_PATH = "/Users/jeremy/Google Drive/datasets/fashion-dataset/"
print(os.listdir(DATASET_PATH))

['embeddings.tsv', '.DS_Store', 'images.csv', 'images', 'styles', 'styles.csv', '.ipynb_checkpoints', 'embeddings.csv', 'resnet50-embeddings.pkl']


In [3]:
df = pd.read_csv(DATASET_PATH + "styles.csv", nrows=5000, error_bad_lines=False)
df['image'] = df.apply(lambda row: str(row['id']) + ".jpg", axis=1)
df = df.reset_index(drop=True)
df.head(10)

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,image
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011,Casual,Turtle Check Men Navy Blue Shirt,15970.jpg
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012,Casual,Peter England Men Party Blue Jeans,39386.jpg
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016,Casual,Titan Women Silver Watch,59263.jpg
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011,Casual,Manchester United Men Solid Black Track Pants,21379.jpg
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012,Casual,Puma Men Grey T-shirt,53759.jpg
5,1855,Men,Apparel,Topwear,Tshirts,Grey,Summer,2011,Casual,Inkfruit Mens Chain Reaction T-shirt,1855.jpg
6,30805,Men,Apparel,Topwear,Shirts,Green,Summer,2012,Ethnic,Fabindia Men Striped Green Shirt,30805.jpg
7,26960,Women,Apparel,Topwear,Shirts,Purple,Summer,2012,Casual,Jealous 21 Women Purple Shirt,26960.jpg
8,29114,Men,Accessories,Socks,Socks,Navy Blue,Summer,2012,Casual,Puma Men Pack of 3 Socks,29114.jpg
9,30039,Men,Accessories,Watches,Watches,Black,Winter,2016,Casual,Skagen Men Black Watch,30039.jpg


In [4]:
# Load ResNet-50 model pretrained on Imagenet without the classifying layers on top.
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add Global Max Pooling layer on top
model = keras.Sequential([
    base_model,
    GlobalMaxPooling2D()
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_max_pooling2d (Global (None, 2048)              0         
Total params: 23,587,712
Trainable params: 23,534,592
Non-trainable params: 53,120
_________________________________________________________________


In [5]:
def get_img_path(img):
  return DATASET_PATH + "images/" + img

In [6]:
def extract_embeddings(img_name, model):
  input_shape = (224, 224, 3)
  img = image.load_img(get_img_path(img_name), target_size=(input_shape[0], input_shape[1]))
  img_array = image.img_to_array(img)
  expanded_img_array = np.expand_dims(img_array, axis=0)
  preprocessed_img = preprocess_input(expanded_img_array)
  embeddings = model.predict(preprocessed_img)
  return embeddings.reshape(-1)

In [8]:
emb = extract_embeddings(df.iloc[0].image, model)
emb.shape

(2048,)

In [9]:
img_array = cv2.imread(get_img_path(df.iloc[282].image))
plt.imshow(cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB))
print(img_array.shape)
print(emb)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(1440, 1080, 3)
[ 5.661311   2.366158   0.        ...  1.3138627  0.        10.820534 ]


In [ ]:
# Create embeddings and store them into dataframe
df_sample      = df#.sample(10)
map_embeddings = df_sample['image'].apply(lambda img: extract_embeddings(img, model))
df_embs        = map_embeddings.apply(pd.Series)

In [ ]:
# Serialize dataframe to pickle file
df_embs.to_pickle(DATASET_PATH + "resnet50-embeddings.pkl")

In [7]:
df_embs = pickle.load(open(DATASET_PATH + "resnet50-embeddings.pkl", "rb"))

In [8]:
# Filter only embeddings with master category 'Apparel' and save indices to filter for them later
df_filtered = df.loc[df.masterCategory == 'Apparel']
df_filtered_idx = df_filtered.index.values.tolist()
df_filtered = df_filtered.reset_index(drop=True)
df_filtered.masterCategory.unique(), df_filtered.shape, len(df_filtered_idx)

(array(['Apparel'], dtype=object), (2297, 11), 2297)

In [9]:
embs_filtered = []
for i, row in df_embs.iterrows():
    if i in  df_filtered_idx:
        embs_filtered.append(row)
df_embs_filtered = pd.DataFrame(embs_filtered)
df_embs_filtered = df_embs_filtered.reset_index(drop=True)
df_embs_filtered

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,5.661316,2.366165,0.000000,3.255730,0.845208,2.673841,8.780870,6.280786,0.787188,1.587657,...,2.463610,0.000000,1.825994,20.485268,3.902863,0.000000,8.225969,1.313863,0.000000,10.820526
1,5.790417,9.593648,0.000000,9.431670,0.287235,0.259945,17.028748,3.037453,0.136341,0.000000,...,1.670570,0.013891,10.338681,4.784722,0.189140,0.000000,7.739144,0.372693,0.000000,8.903802
2,5.882203,1.958606,0.000000,12.519882,0.000000,0.000000,9.383594,3.781614,0.000000,1.490617,...,17.825478,0.000000,3.592507,2.042305,0.000000,0.000000,6.656779,1.457314,1.694204,1.403327
3,0.245968,17.383430,0.493824,3.374468,2.567368,0.000000,6.482537,5.862639,0.000000,3.164866,...,4.464024,0.447075,1.448143,12.884138,0.000000,0.397501,5.028075,1.939697,0.000000,11.485400
4,0.028546,18.567099,0.000000,0.757250,1.504949,0.000000,1.982024,2.963777,1.072765,4.539300,...,3.247082,4.273134,3.088643,16.997053,0.000000,0.000000,2.258095,1.595798,0.000000,7.674047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2292,3.134137,12.337814,3.281830,0.000000,6.123401,6.273666,3.345068,4.230134,2.082114,2.691854,...,5.392367,2.139013,7.540308,5.394151,6.070920,0.000000,0.000000,0.000000,0.000000,10.168349
2293,2.800386,19.866081,3.215006,2.487933,0.147357,0.000000,7.958175,5.669247,0.706661,4.877097,...,10.292748,0.000000,0.738173,16.109261,0.383140,0.000000,4.432935,2.898650,0.416252,12.830476
2294,4.801172,16.536064,2.132150,2.630906,1.190370,0.243107,1.454644,3.500423,0.627453,5.221931,...,0.000000,1.886256,3.294083,17.186272,0.000000,2.014549,1.663332,2.118907,0.000000,6.815829
2295,3.184294,14.538852,6.306130,0.000000,3.361131,6.701039,0.729837,0.295319,0.252367,3.175583,...,0.000000,1.471544,1.482233,22.654318,2.425646,12.427614,8.393599,0.000000,9.616654,3.375090


In [10]:
# Perform PCA over the embeddings to reduce dimensionality before applying t-sne
num_feature_dimensions = 2  # Set the number of embedding dimensions
pca = PCA(n_components = num_feature_dimensions)
embs_compressed = pca.fit_transform(df_embs_filtered)
df_embs_filtered_compressed = pd.DataFrame(embs_compressed)
df_embs_filtered_compressed

,0,1
0,-64.281189,-0.382128
1,-50.859043,-78.791481
2,-21.894369,-69.084740
3,-52.814461,-30.695751
4,-26.194759,-12.308769
...,...,...
2292,99.575554,-30.713846
2293,-26.236422,-12.102118
2294,-29.318714,36.647404
2295,54.131573,63.903694


In [11]:
df_embs.head()

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,5.661316,2.366165,0.000000,3.255730,0.845208,2.673841,8.780870,6.280786,0.787188,1.587657,...,2.463610,0.000000,1.825994,20.485268,3.902863,0.000000,8.225969,1.313863,0.000000,10.820526
1,5.790417,9.593648,0.000000,9.431670,0.287235,0.259945,17.028748,3.037453,0.136341,0.000000,...,1.670570,0.013891,10.338681,4.784722,0.189140,0.000000,7.739144,0.372693,0.000000,8.903802
2,0.000000,3.162254,0.484691,0.172299,2.609548,3.840852,3.426139,0.253253,1.098907,1.418282,...,0.434926,14.092834,3.018744,2.134506,2.549062,0.377385,6.180348,0.636620,14.018165,11.415048
3,5.882203,1.958606,0.000000,12.519882,0.000000,0.000000,9.383594,3.781614,0.000000,1.490617,...,17.825478,0.000000,3.592507,2.042305,0.000000,0.000000,6.656779,1.457314,1.694204,1.403327
4,0.245968,17.383430,0.493824,3.374468,2.567368,0.000000,6.482537,5.862639,0.000000,3.164866,...,4.464024,0.447075,1.448143,12.884138,0.000000,0.397501,5.028075,1.939697,0.000000,11.485400


In [12]:
df_filtered[df_filtered['season'].isnull().values == True]

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,image


In [13]:
# ...and override if there are any,
df.at[282, 'season'] = "Summer"

In [14]:
for i, row in df_filtered.iterrows():
    if df_filtered.at[i, 'season'] == "Spring":
        df_filtered.at[i, 'season'] = "Summer"
    elif df_filtered.at[i, 'season'] == "Fall":
        df_filtered.at[i, 'season'] = "Winter"
df_filtered.season.unique()

array(['Winter', 'Summer'], dtype=object)

In [15]:
df_filtered

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,image
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Winter,2011,Casual,Turtle Check Men Navy Blue Shirt,15970.jpg
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012,Casual,Peter England Men Party Blue Jeans,39386.jpg
2,21379,Men,Apparel,Bottomwear,Track Pants,Black,Winter,2011,Casual,Manchester United Men Solid Black Track Pants,21379.jpg
3,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012,Casual,Puma Men Grey T-shirt,53759.jpg
4,1855,Men,Apparel,Topwear,Tshirts,Grey,Summer,2011,Casual,Inkfruit Mens Chain Reaction T-shirt,1855.jpg
...,...,...,...,...,...,...,...,...,...,...,...
2292,41001,Girls,Apparel,Bottomwear,Shorts,Blue,Summer,2012,Casual,Gini and Jony Girls Woven Blue Shorts,41001.jpg
2293,11771,Men,Apparel,Topwear,Tshirts,Maroon,Winter,2011,Casual,Lee Men Printed Maroon Tshirts,11771.jpg
2294,25544,Women,Apparel,Topwear,Tshirts,Navy Blue,Summer,2012,Casual,Wrangler Women Sunrise Navy Blue T-shirt,25544.jpg
2295,57958,Women,Apparel,Saree,Sarees,Green,Summer,2012,Ethnic,Prafful Green Printed Sari,57958.jpg


In [16]:
# Application of SVM to create the axis for a given feature
X = df_embs_filtered
y = df_filtered["season"]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
svm_clf = LinearSVC(C=1, max_iter=100000)
svm_clf.fit(X_scaled, y)

LinearSVC(C=1, max_iter=100000)

In [17]:
df_filtered['x'] = df_embs_filtered_compressed[0]
df_filtered['y'] = df_embs_filtered_compressed[1]

In [18]:
def create_user_marker(x, y):
    """Creates and returns a user marker on a specific position"""
    user_marker = plt.plot(x, y, 'yo', markersize=10)
    user_position = np.array([x, y])
    return user_marker, user_position

In [19]:
def get_updated_user_pos(change, axis_x, axis_y):
    """
    Computes the index of the element that is closest to the given change value in axis_y.
    Returns the new user position.
    """
    index = np.where(np.isclose(axis_x, min(axis_x, key=lambda x:abs(x-change))))
    return axis_x[index[0][0]], axis_y[index[0][0]]

In [20]:
def update_user_position_2D(change):
    """Update the user position after the slider value has changed"""
    new_x, new_y = get_updated_user_pos(change.new, db_xx, ortho_db_yy)
    user_marker[0].set_data(new_x, new_y)
    new_user_pos = np.array([new_x, new_y])
    user_positon = new_user_pos
    nearest_neighbour, nearest_neighbour_pos = get_nearest_neighbour(new_user_pos, df_filtered)
    annotate_nearest_neighbour(nearest_neighbour, nearest_neighbour_pos, ax, df_filtered)
    plt.title('Nearest Embedding: {} with season: {}, pos: {}'.format(nearest_neighbour, df_filtered.loc[df_filtered['id'] == nearest_neighbour].season.values[0], user_positon))
    fig.canvas.draw()
    fig.canvas.flush_events()

In [21]:
def calc_svm_decision_boundary(svm_clf, xmin, xmax):
    """Compute a decision boundary and ret"""
    w = svm_clf.coef_[0]
    b = svm_clf.intercept_[0]
    xx = np.linspace(xmin, xmax, 200)
    yy = -w[0]/w[1] * xx - b/w[1]
    return xx, yy

In [22]:
def get_nearest_neighbour(user_position, df):
   nearest_neighbour = None
   nearest_neighbour_pos = None
   smallest_dist = sys.maxsize
   for row in df.itertuples():
      embedding_position = np.array([row.x, row.y])
      dist = norm(user_position - embedding_position)
      if dist < smallest_dist:
         smallest_dist = dist
         nearest_neighbour = row.id
         nearest_neighbour_pos = embedding_position
   return nearest_neighbour, nearest_neighbour_pos

In [23]:
def highlight_nearest_neighbour(id, df):
    x_nn = df.loc[df['id'] == id].x.values[0]
    y_nn = df.loc[df['id'] == id].y.values[0]
    plt.scatter(x=x_nn, y=y_nn, color='r')

In [24]:
def annotate_nearest_neighbour(nearest_neighbour, nearest_neighbour_pos, ax, df):
    if ax.artists != []:
        ax.artists[0].remove()
    arr_img = plt.imread(get_img_path(df.loc[df['id'] == nearest_neighbour].image.values[0]))
    imagebox = OffsetImage(arr_img, zoom=0.025)
    imagebox.image.axes = ax
    ab = AnnotationBbox(imagebox, nearest_neighbour_pos, xybox=(-20, 40), xycoords='data', boxcoords="offset points", arrowprops=dict(arrowstyle="->"))
    ax.add_artist(ab)

In [25]:
df_filtered.loc[df_filtered['id'] == 19787]

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,image,x,y
1625,19787,Men,Apparel,Topwear,Tshirts,White,Winter,2016,Casual,U.S. Polo Assn. Men Solid White Tshirt,19787.jpg,-37.939262,-36.072918


In [26]:
node = df_filtered.sample()
node['x']

100   -26.977482
Name: x, dtype: float32

In [40]:
starting_node.x.values[0]

93.98002

In [39]:
db_xx

array([-35.        , -34.64824121, -34.29648241, -33.94472362,
       -33.59296482, -33.24120603, -32.88944724, -32.53768844,
       -32.18592965, -31.83417085, -31.48241206, -31.13065327,
       -30.77889447, -30.42713568, -30.07537688, -29.72361809,
       -29.3718593 , -29.0201005 , -28.66834171, -28.31658291,
       -27.96482412, -27.61306533, -27.26130653, -26.90954774,
       -26.55778894, -26.20603015, -25.85427136, -25.50251256,
       -25.15075377, -24.79899497, -24.44723618, -24.09547739,
       -23.74371859, -23.3919598 , -23.04020101, -22.68844221,
       -22.33668342, -21.98492462, -21.63316583, -21.28140704,
       -20.92964824, -20.57788945, -20.22613065, -19.87437186,
       -19.52261307, -19.17085427, -18.81909548, -18.46733668,
       -18.11557789, -17.7638191 , -17.4120603 , -17.06030151,
       -16.70854271, -16.35678392, -16.00502513, -15.65326633,
       -15.30150754, -14.94974874, -14.59798995, -14.24623116,
       -13.89447236, -13.54271357, -13.19095477, -12.83

In [59]:
db_xx + starting_node.x.values[0]

array([-80.92983627, -80.57807748, -80.22631869, -79.87455989,
       -79.5228011 , -79.1710423 , -78.81928351, -78.46752472,
       -78.11576592, -77.76400713, -77.41224833, -77.06048954,
       -76.70873075, -76.35697195, -76.00521316, -75.65345436,
       -75.30169557, -74.94993678, -74.59817798, -74.24641919,
       -73.89466039, -73.5429016 , -73.19114281, -72.83938401,
       -72.48762522, -72.13586642, -71.78410763, -71.43234884,
       -71.08059004, -70.72883125, -70.37707245, -70.02531366,
       -69.67355487, -69.32179607, -68.97003728, -68.61827848,
       -68.26651969, -67.9147609 , -67.5630021 , -67.21124331,
       -66.85948451, -66.50772572, -66.15596693, -65.80420813,
       -65.45244934, -65.10069054, -64.74893175, -64.39717296,
       -64.04541416, -63.69365537, -63.34189657, -62.99013778,
       -62.63837899, -62.28662019, -61.9348614 , -61.5831026 ,
       -61.23134381, -60.87958502, -60.52782622, -60.17606743,
       -59.82430864, -59.47254984, -59.12079105, -58.76

In [66]:
from ipywidgets import AppLayout, FloatSlider
from matplotlib.offsetbox import (AnnotationBbox, OffsetImage, TextArea)

plt.ioff()

fig, ax = plt.subplots(figsize=(15,7))
fig.canvas.header_visible = False
fig.canvas.layout.min_height = '400px'

# Create Scatterplot of filtered dataset colored by season feature
sns.scatterplot(x="x", y="y",
                  hue="season",
                  data=df_filtered,
                  legend="full",
                  alpha=0.8)

# Computes the decision boundary of a trained classifier
db_xx, db_yy = calc_svm_decision_boundary(svm_clf, -35, 35)

# Rotate the decision boundary 90° to get perpendicular axis
neg_yy = np.negative(db_yy) 

neg_slope = -1 / -svm_clf.coef_[0][0]
bias = svm_clf.intercept_[0] / svm_clf.coef_[0][1]
ortho_db_yy = neg_slope * db_xx - bias

# Plot the axes
plt.plot(db_xx, db_yy, "k-", linewidth=1)
plt.plot(db_xx, ortho_db_yy, "r-", linewidth=1)
#plt.plot(neg_yy, db_xx, "g-", linewidth=2)

# Choose a random starting position and initialize user marker on that position
rand_idx = random.choice(range(len(db_xx)))
x = db_xx[rand_idx]
y = ortho_db_yy[rand_idx]
user_marker, user_positon = create_user_marker(x, y)

# Compute the nearest neighbour and annotate it with its respective image
nearest_neighbour, nearest_neighbour_pos = get_nearest_neighbour(user_positon, df_filtered)
annotate_nearest_neighbour(nearest_neighbour, nearest_neighbour_pos, ax, df_filtered)

plt.title('Nearest Embedding: {} with season: {}, pos: {}'.format(nearest_neighbour, df_filtered.loc[df_filtered['id'] == nearest_neighbour].season.values[0], user_positon))

# Create Slider to interact with the plot
slider = FloatSlider(
    orientation="horizontal",
    description="x-Position:",
    value=user_positon[0],
    min=min(db_xx),
    max=max(db_xx)
)
slider.layout.margin = '0px 30% 0px 30%'
slider.layout.width = '25%'

slider.observe(update_user_position_2D, names='value')

AppLayout(
    center=fig.canvas,
    footer=slider,
    pane_heights=[0, 6, 1]
)

AppLayout(children=(FloatSlider(value=-32.537688442211056, description='x-Position:', layout=Layout(grid_area=…